In [12]:
import pygame as pg
import random

In [28]:
flappy_bird()

In [27]:
def flappy_bird():
    def create_text(text,fg,bg,font = 'calibri',size=20):
        Font = pg.font.SysFont(font,size)
        Text = Font.render(text,True,fg,bg)
        return Text
    def in_rect(pos,rect):
        return pos[0]>=rect[0] and pos[0]<=rect[0]+rect[2] and pos[1]>=rect[1] and pos[1]<=rect[1]+rect[3]

    pg.init()
    screen = pg.display.set_mode((600,500))
    pg.display.set_caption('Flappy Bird')
    gamespace = [0,100,600,500]
    
    white = (255,255,255)

    score_board = pg.Surface((600,100))
    score_board_bg = pg.image.load('rocky_bg.jpg')
    score_board.blit(score_board_bg,(0,0))
    score_board_rect = score_board.get_rect()
    pg.draw.rect(score_board,(150,150,150),(gamespace[0],gamespace[1]-10)+(score_board_rect.width,10))

    highscore_value = 0
    score_value = 0

    highscore_text = create_text(str(highscore_value),white,(0,0,0))
    highscore_rect = highscore_text.get_rect()
    highscore_rect.center = (score_board_rect.width*7/8,score_board_rect.height/4)
    score_text = create_text(str(score_value),white,(0,0,0))
    score_rect = score_text.get_rect()
    score_rect.center = (score_board_rect.width*7/8,score_board_rect.height*3/4)

    flappy_bird= pg.image.load('flappy_bird.png')
    flappy_bird_rect = flappy_bird.get_rect()
    flappy_bird_initial_position = ((gamespace[0]+gamespace[2])/3,(gamespace[1]+gamespace[3])/2)
    flappy_bird_rect.center = flappy_bird_initial_position

    trophy = pg.image.load('trophy.png')
    trophy_rect = trophy.get_rect()
    trophy_rect.center = (score_board.get_width()*13/16,score_board.get_height()*1/4)
    small_pipe = pg.image.load('small_pipe.png')
    small_pipe_rect = small_pipe.get_rect()
    small_pipe_rect.centerx = trophy_rect.centerx
    small_pipe_rect.centery = score_board.get_height()*3/4

    pipeup = pg.image.load('pipe.png')
    pipeup_rect = pipeup.get_rect()
    pipeup_rect.right = gamespace[2]
    pipeup_rect.bottom = gamespace[3]

    pipedown = pg.transform.flip(pipeup.copy(),False,True)
    pipedown_rect = pipedown.get_rect()
    pipedown_rect.right = gamespace[2]
    pipedown_rect.top = gamespace[1]

    pipes = (pipeup,pipedown)
    pipe_rects = (pipeup_rect,pipedown_rect)
    pipe_initial_position = gamespace[2]

    flappy_bg = pg.image.load('flappy_bg.png')
    flappy_bg_rect = flappy_bg.get_rect()
    flappy_bg_rect.topleft = (gamespace[0],gamespace[1])

    game_over = pg.image.load('game_over.png')

    score_board.blit(highscore_text,highscore_rect)
    score_board.blit(score_text,score_rect)

    score_board.blit(trophy,trophy_rect)
    score_board.blit(small_pipe,small_pipe_rect)

    screen.blit(score_board,(0,0))

    screen.blit(flappy_bg,gamespace[0:2])

    screen.blit(flappy_bird,flappy_bird_rect)

    def start_game():
        def highscore_set_routine():
            nonlocal highscore_value,score_value,highscore_rect
            if highscore_value<score_value:
                highscore_value = score_value
            text = create_text(str(highscore_value),white,(0,0,0))
            text_rect = text.get_rect()
            text_rect.topleft = highscore_rect.topleft
            highscore_rect = text_rect.copy()
            replacewith_subimage(score_board,flappy_bg,flappy_bg_rect,highscore_rect)
            score_board.blit(text,highscore_rect)
            screen.blit(score_board,(0,0))
            score_reset_routine()
        def score_reset_routine():
            nonlocal score_value,score_rect
            score_value=0
            text = create_text(str(score_value),white,(0,0,0))
            text_rect = text.get_rect()
            text_rect.topleft = score_rect.topleft
            score_rect = text_rect.copy()
            replacewith_subimage(score_board,flappy_bg,flappy_bg_rect,score_rect)
            score_board.blit(text,score_rect)
            screen.blit(score_board,(0,0))
        def score_increment_routine():
            nonlocal score_value,score_rect
            score_value+=1
            text = create_text(str(score_value),white,(0,0,0))
            text_rect = text.get_rect()
            text_rect.topleft = score_rect.topleft
            score_rect = text_rect.copy()
            replacewith_subimage(score_board,flappy_bg,flappy_bg_rect,score_rect)
            score_board.blit(text,score_rect)
            screen.blit(score_board,(0,0))
        def replay_routine():
            nonlocal pipe_list,void_list,pipe_Dx,start_flag,current_pointer
            start_flag = False
            current_pointer = 0
            screen.blit(flappy_bg,gamespace[:2])
            flappy_bird_rect.center = flappy_bird_initial_position
            pipe_list = []
            void_list = []
            pipe_Dx = pipe_separation
        def game_over_routine():
            nonlocal running
            pg.display.update()
            pg.time.delay(500)
            game_over_rect = game_over.get_rect()
            game_over_rect.center = ((gamespace[2]+gamespace[0])/2,(gamespace[3]+gamespace[1])/2)
            screen.blit(game_over,game_over_rect)
            pg.display.update()
            pg.time.delay(500)
            pg.event.get()
            run = True
            while run:
                for event in pg.event.get():
                    if event.type==pg.QUIT:
                        running = False
                        return
                    elif event.type == pg.KEYDOWN:
                        if event.key == pg.K_SPACE:
                            highscore_set_routine()
                            replay_routine()
                            return
                pg.display.update()
        def generate_void(Range,height):
            number = random.randrange(Range[0],Range[1])
            return (number,number+height)
        def generate_pipes(void_range,pipes,pipe_rects):
            upimage = pipes[0].subsurface((0,0,pipe_rects[0].width,gamespace[3]-void_range[1]))
            downimage = pipes[1].subsurface((0,pipe_rects[1].height-(void_range[0]-gamespace[1]),
                                             pipe_rects[1].width,void_range[0]-gamespace[1]))
            return (upimage,downimage)
        def replacewith_subimage(surface,image,image_rect,rect):
            surface.blit(image,rect,(rect.left-image_rect.left,rect.top-image_rect.top)+(rect.width,rect.height))
        def set_pipe_separation():
            nonlocal pipe_separation
            pipe_separation =  random.randrange(pipe_separation_range[0],pipe_separation_range[1])

        frame_rate = 110
        clock = pg.time.Clock()
        flappy_bird_Vy=0
        fb_boundary_apx = ((flappy_bird_rect.height/100)*10,(flappy_bird_rect.width/100)*40)
        pipe_Vx = 230
        jump_height = 0.35
        g = 9.80665
        pipe_list=[]
        void_list = []
        pipe_separation_range = (200,500)
        current_pointer = 0
        pipe_separation = 300
        void_separation = 100
        pipe_Dx = pipe_separation
        start_flag = False
        running = True
        while running:
            clock.tick(frame_rate)
            if start_flag:
                flappy_bird_change = (flappy_bird_Vy*(1/frame_rate)+g*((1/frame_rate)**2)/2)*200
                pipe_change = -(pipe_Vx*(1/frame_rate))
                pipe_Dx+=abs(pipe_change)
                flappy_bird_Vy += g*(1/frame_rate)
                if pipe_Dx>=pipe_separation:
                    void_list.append(generate_void((gamespace[1]+50,gamespace[3]-(void_separation+50)),void_separation))
                    pipe_list.append(generate_pipes(void_list[-1],pipes,pipe_rects))
                    rect1 = pipe_list[-1][0].get_rect()
                    rect2 = pipe_list[-1][1].get_rect()
                    rect1.bottom = gamespace[3]
                    rect1.left = pipe_initial_position
                    rect2.top = gamespace[1]
                    rect2.left = pipe_initial_position
                    pipe_list[-1]+=(rect1,rect2)
                    pipe_Dx = 0
                    set_pipe_separation()
                if pipe_list[0][3].right<gamespace[0]:
                    pipe_list.pop(0)
                    void_list.pop(0)
                    current_pointer-=1
                if flappy_bird_rect.top+flappy_bird_change<=gamespace[1]:
                    flappy_bird_Vy = 0
                else:
                    #screen.blit(flappy_bg,flappy_bird_rect,(flappy_bird_rect.left-gamespace[0],flappy_bird_rect.top-gamespace[1])+
                               #(flappy_bird_rect.width,flappy_bird_rect.height))
                    replacewith_subimage(screen,flappy_bg,flappy_bg_rect,flappy_bird_rect)
                    for i in pipe_list:
                        #screen.blit(flappy_bg,i[2],(i[2].left-gamespace[0],i[2].top-gamespace[1])+
                                    #(i[2].width,i[2].height))
                        replacewith_subimage(screen,flappy_bg,flappy_bg_rect,i[2])
                        #screen.blit(flappy_bg,i[3],(i[3].left-gamespace[0],i[3].top-gamespace[1])+
                                    #(i[3].width,i[3].height))
                        replacewith_subimage(screen,flappy_bg,flappy_bg_rect,i[3])
                    flappy_bird_rect.move_ip(0,flappy_bird_change)
                    for i in pipe_list:
                        i[2].move_ip(pipe_change,0)
                        i[3].move_ip(pipe_change,0)
            for i in pipe_list:
                screen.blit(i[0],i[2])
                screen.blit(i[1],i[3])
            screen.blit(flappy_bird,flappy_bird_rect)

            if start_flag and pipe_list[current_pointer][2].left<flappy_bird_rect.right-fb_boundary_apx[1]:
                uppertouch = flappy_bird_rect.top+fb_boundary_apx[0]<void_list[current_pointer][0]
                lowertouch = flappy_bird_rect.bottom-fb_boundary_apx[0]>void_list[current_pointer][1]
                if uppertouch or lowertouch:
                    game_over_routine()
            if start_flag and pipe_list[current_pointer][2].right<flappy_bird_rect.left+fb_boundary_apx[1]:
                current_pointer+=1
                score_increment_routine()
            if start_flag and flappy_bird_rect.top>=gamespace[3]:
                game_over_routine()
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    running = False
                elif event.type == pg.KEYDOWN:
                    if event.key == pg.K_SPACE:
                        start_flag = True
                        flappy_bird_Vy = -(2*g*jump_height)**(1/2)
                else:
                    pass
            pg.display.flip()
        pg.quit()
    start_game()